In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
df=pd.read_csv("IRIS.csv")

In [3]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [4]:
df1=df.drop('species',axis=1)

In [5]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [40]:
def relu(x):
    return (abs(x.astype(float))+x.astype(float))/2
def softmax(x):
    exp_scores = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
def derivative(x):
    y=x
    np.piecewise(y,[relu(y)==0,relu(y)==y],[0,1])
    return y
        

In [41]:
X = df.drop('species', axis=1).values
Y = df['species'].values.reshape(-1, 1)

In [42]:
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(Y)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

In [44]:
input_size = X.shape[1]
hidden1_size = 7
hidden2_size = 4
output_size = y_onehot.shape[1]
learning_rate = 0.01
epochs = 5

In [45]:
np.random.seed(40)
weights_input_hidden1 = np.random.randn(input_size, hidden1_size)
bias_input_hidden1 = np.random.randn(1, hidden1_size)
weights_hidden1_hidden2 = np.random.randn(hidden1_size, hidden2_size)
bias_hidden1_hidden2 = np.random.randn(1, hidden2_size)
weights_hidden2_output = np.random.randn(hidden2_size, output_size)
bias_hidden2_output = np.random.randn(1, output_size)

In [46]:
y_train

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1

In [47]:
for epoch in range(epochs):

    hidden1_input = np.dot(X_train, weights_input_hidden1) + bias_input_hidden1
    hidden1_output = relu(hidden1_input)
    hidden2_input = np.dot(hidden1_output, weights_hidden1_hidden2) + bias_hidden1_hidden2
    hidden2_output = relu(hidden2_input)
    output_layer_input = np.dot(hidden2_output, weights_hidden2_output) + bias_hidden2_output
    output = softmax(output_layer_input)
   
    
    error = y_train - output
    d_output = error * derivative(output)
    error_hidden2_layer = d_output.dot(weights_hidden2_output.T)
    d_hidden2_output = error_hidden2_layer * derivative(hidden2_output)
    error_hidden1_layer = d_hidden2_output.dot(weights_hidden1_hidden2.T)
    d_hidden1_output = error_hidden1_layer * derivative(hidden1_output)
    
    weights_hidden2_output += hidden2_output.T.dot(d_output) * learning_rate
    bias_hidden2_output += np.sum(d_output, axis=0, keepdims=True) * learning_rate
    weights_hidden1_hidden2 += hidden1_output.T.dot(d_hidden2_output) * learning_rate
    bias_hidden1_hidden2 += np.sum(d_hidden2_output, axis=0, keepdims=True) * learning_rate
    weights_input_hidden1 += X_train.T.dot(d_hidden1_output) * learning_rate
    bias_input_hidden1 += np.sum(d_hidden1_output, axis=0, keepdims=True) * learning_rate

/tmp/ipykernel_7173/4165530556.py:16: RuntimeWarning: overflow encountered in multiply
  d_hidden1_output = error_hidden1_layer * derivative(hidden1_output)


In [48]:
hidden1_input = np.dot(X_test, weights_input_hidden1) + bias_input_hidden1
hidden1_output = relu(hidden1_input)
hidden2_input = np.dot(hidden1_output, weights_hidden1_hidden2) + bias_hidden1_hidden2
hidden2_output = relu(hidden2_input)
output_layer_input = np.dot(hidden2_output, weights_hidden2_output) + bias_hidden2_output
output = softmax(output_layer_input)
predicted_labels = np.argmax(output, axis=1)
true_labels = np.argmax(y_test, axis=1)

/tmp/ipykernel_7173/2370352661.py:2: RuntimeWarning: invalid value encountered in add
  return (abs(x.astype(float))+x.astype(float))/2


In [49]:
predicted_labels


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [50]:
accuracy = np.mean(predicted_labels == true_labels)
print("Accuracy:", accuracy)


Accuracy: 0.3333333333333333
